# Дообучение модели GPT-3 на корпусе текстов юридического характера

Имеем 20654 судебных решений.

Отбираем случайным образом по 1 000 для каждого раздела: 
* административные дела
* гражданские дела
* уголовные дела

In [10]:
import os

In [126]:
# получаем все идентификаторы из decisions__data
parsed_decisions_ids = sorted(list(map(lambda x: int(x.replace('.txt', '')),
                           os.listdir('decisions__data'))))
len(parsed_decisions_ids)

5157

In [100]:
parsed_decisions_ids[:5]

[1, 2, 3, 4, 5]

In [ ]:
# определяем принадлежность каждого решения к разделам

In [18]:
import sqlite3

In [19]:
con = sqlite3.connect("sudactru_nl.db")
cur = con.cursor()

In [24]:
res = cur.execute(
    '''
    SELECT 
        *
    FROM 
        sections 
    '''
)
res.fetchall()

[(1,
  'По административным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/'),
 (2,
  'По гражданским делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/'),
 (3,
  'По уголовным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/')]

In [107]:
def get_label_by_decision_id(decision_id):
    res = cur.execute(
    '''
    SELECT 
        subsection_id 
    FROM 
        decisions 
    WHERE id = ?
    ''', 
      (decision_id,)
    )
    subsection_id = res.fetchone()[0]
    
    res = cur.execute(
        '''
        SELECT 
            section_id
        FROM 
            subsections 
        WHERE id = ?
        ''', 
      (subsection_id,)
    )
    section_id = res.fetchone()[0]
    section_id
    
    label_map = {
        1: 'administrative',
        2: 'civil',
        3: 'criminal'
    }
    
    return label_map[section_id]

In [116]:
get_label_by_decision_id(parsed_decisions_ids[0])

'criminal'

In [117]:
get_label_by_decision_id(1058)

'administrative'

In [122]:
# считаем статистику по всему датасету
res = cur.execute(
    '''
    SELECT 
        --d.id, 
        --d.name,
        s.id AS section_id,
        s.name,
        COUNT(*)
    FROM 
        decisions AS d
    INNER JOIN subsections AS ss ON d.subsection_id = ss.id
    INNER JOIN sections AS s ON s.id = ss.section_id
    GROUP BY
        section_id
    '''
)
res.fetchall()[:25]

[(1, 'По административным делам', 13668),
 (2, 'По гражданским делам', 166),
 (3, 'По уголовным делам', 37470)]

In [110]:
# считаем статистику, сколько уже есть спаршенных данных по каждому разделу

In [127]:
labels_stats = {
    'administrative': 0,
    'civil': 0,
    'criminal': 0
}

In [128]:
from tqdm import tqdm

In [129]:
for decision_id in tqdm(parsed_decisions_ids):
    label = get_label_by_decision_id(decision_id)
    labels_stats[label] += 1

100%|████████████████████████████████████████████████████████████████████████| 5157/5157 [00:00<00:00, 37259.74it/s]


In [130]:
labels_stats

{'administrative': 1616, 'civil': 0, 'criminal': 3541}

In [123]:
# res = cur.execute("SELECT * FROM decisions LIMIT 5;")
# res.fetchall()

In [ ]:
# разделяем данные на папки с названиями метов

In [131]:
import shutil

In [132]:
for decision_id in tqdm(parsed_decisions_ids):
    label = get_label_by_decision_id(decision_id)
    
    shutil.copyfile(f'decisions__data/{decision_id}.txt', 
                    f'{label}/{decision_id}.txt')

100%|█████████████████████████████████████████████████████████████████████████| 5157/5157 [00:02<00:00, 2380.37it/s]


In [ ]:
# выбираем случайным образом по 1 000 элементов каждого раздела

In [133]:
administrative = list(map(lambda x: int(x.replace('.txt', '')),
                           os.listdir('administrative')))
administrative

[3644,
 1053,
 3122,
 4881,
 1047,
 3650,
 3888,
 9433,
 2566,
 10021,
 5209,
 9427,
 4856,
 276,
 538,
 3877,
 5037,
 5023,
 3452,
 2994,
 4473,
 4301,
 2943,
 706,
 712,
 3491,
 10592,
 869,
 3532,
 10357,
 2610,
 699,
 2604,
 4513,
 3297,
 5194,
 3283,
 128,
 882,
 3730,
 4739,
 2348,
 3042,
 10155,
 4063,
 4711,
 3718,
 470,
 316,
 302,
 4936,
 4088,
 464,
 9590,
 9591,
 4937,
 303,
 465,
 4089,
 471,
 317,
 3719,
 4062,
 10154,
 3043,
 2349,
 4738,
 883,
 129,
 5195,
 101,
 3282,
 3296,
 2605,
 698,
 4512,
 10356,
 3533,
 868,
 713,
 707,
 2942,
 2771,
 4472,
 2995,
 3453,
 5022,
 3321,
 2981,
 5036,
 3876,
 539,
 277,
 4857,
 9340,
 9426,
 2573,
 5208,
 10020,
 2567,
 9432,
 3889,
 1046,
 1052,
 3645,
 3123,
 1044,
 4882,
 3121,
 3647,
 1050,
 9424,
 10022,
 2565,
 275,
 4699,
 4855,
 3874,
 4869,
 3451,
 2997,
 5034,
 2983,
 3323,
 2773,
 4302,
 711,
 3492,
 2954,
 705,
 3531,
 2607,
 10354,
 103,
 5197,
 881,
 2808,
 2439,
 4048,
 3041,
 10618,
 4712,
 1118,
 10156,
 4060,
 467

In [134]:
criminal = list(map(lambda x: int(x.replace('.txt', '')),
                           os.listdir('criminal')))
criminal

[4895,
 3136,
 4659,
 10009,
 4117,
 3678,
 4671,
 9355,
 289,
 9341,
 2214,
 4665,
 2572,
 10035,
 504,
 1090,
 2599,
 3687,
 262,
 4842,
 3693,
 1084,
 510,
 3863,
 9396,
 3320,
 10579,
 3446,
 2980,
 4329,
 909,
 3334,
 10545,
 10223,
 2764,
 4315,
 921,
 9631,
 935,
 10237,
 2770,
 4467,
 3308,
 10551,
 3485,
 9816,
 9802,
 4498,
 10586,
 3254,
 29,
 5143,
 4249,
 3526,
 2638,
 5157,
 10419,
 3240,
 10431,
 3268,
 4507,
 4261,
 9745,
 15,
 855,
 9751,
 9989,
 4275,
 10343,
 10425,
 2162,
 9037,
 114,
 5180,
 2837,
 672,
 666,
 2823,
 100,
 896,
 10394,
 10380,
 10169,
 1641,
 3056,
 9209,
 9547,
 4705,
 2360,
 2406,
 4077,
 4922,
 3095,
 3081,
 3917,
 9584,
 458,
 3903,
 10182,
 10183,
 3902,
 459,
 9585,
 3916,
 3080,
 3094,
 4923,
 2361,
 4710,
 4076,
 2407,
 9546,
 9220,
 4704,
 9208,
 3731,
 10168,
 3057,
 10381,
 10395,
 897,
 2822,
 667,
 115,
 673,
 2836,
 5181,
 10342,
 4274,
 840,
 9988,
 9750,
 9036,
 2163,
 10424,
 4506,
 3269,
 10430,
 854,
 14,
 9744,
 4260,
 5156,
 26

In [135]:
import random

In [136]:
administrative_1000 = random.sample(administrative, 1000)
administrative_1000[:5]

[3120, 10356, 880, 9588, 31]

In [137]:
criminal_1000 = random.sample(criminal, 1000)
criminal_1000[:5]

[2296, 2664, 1072, 10037, 4664]

In [138]:
for decision_id in tqdm(administrative_1000):    
    shutil.copyfile(
        f'decisions__data/{decision_id}.txt', 
        f'dataset/all/administrative/{decision_id}.txt'
    )

100%|█████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2464.23it/s]


In [139]:
for decision_id in tqdm(criminal_1000):    
    shutil.copyfile(
        f'decisions__data/{decision_id}.txt', 
        f'dataset/all/criminal/{decision_id}.txt'
    )

100%|█████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2350.85it/s]


In [141]:
# разбиваем на обучающую и валидационную выборки
administrative_val_ind = random.sample(administrative_1000, 100)

In [142]:
administrative_train = [
    ind for ind in administrative_1000 if ind not in administrative_val_ind
]
administrative_valid = [
    ind for ind in administrative_1000 if ind in administrative_val_ind
]

In [144]:
for decision_id in tqdm(administrative_train):    
    shutil.copyfile(
        f'decisions__data/{decision_id}.txt', 
        f'dataset/train/administrative/{decision_id}.txt'
    )

100%|███████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 2510.10it/s]


In [145]:
for decision_id in tqdm(administrative_valid):    
    shutil.copyfile(
        f'decisions__data/{decision_id}.txt', 
        f'dataset/test/administrative/{decision_id}.txt'
    )

100%|███████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1967.07it/s]


In [146]:
criminal_val_ind = random.sample(criminal_1000, 100)

In [147]:
criminal_train = [
    ind for ind in criminal_1000 if ind not in criminal_val_ind
]
criminal_valid = [
    ind for ind in criminal_1000 if ind in criminal_val_ind
]

In [148]:
for decision_id in tqdm(criminal_train):    
    shutil.copyfile(
        f'decisions__data/{decision_id}.txt', 
        f'dataset/train/criminal/{decision_id}.txt'
    )

100%|███████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 2331.50it/s]


In [149]:
for decision_id in tqdm(criminal_valid):    
    shutil.copyfile(
        f'decisions__data/{decision_id}.txt', 
        f'dataset/test/criminal/{decision_id}.txt'
    )

100%|███████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1620.80it/s]


In [1]:
# Приводим обучающий датасет к нужному формату:

# Text: "<text>"
# Section: <section_name>

Разделяем выборку на обучающую и тестовую выборку.

Выполняем тонкую настройку модели

Рассчитываем метрики качества

Строим классификацию с применением других моделей:
* gpt с большим количеством параметров
* svm (svc)
* bert

Сравниваем результаты

Незабываем показать кривые обучения

In [9]:
"""
Дело *
Р Е Ш Е Н И Е
И М Е Н Е М Р О С С И Й С К О Й Ф Е Д Е Р А Ц И И
дата *
Поронайский городской суд * в составе председательствующего судьи Домниковой Л.В., при секретаре судебного заседания *2, рассмотрев в открытом судебном заседании гражданское дело по исковому заявлению *1 к Министерству финансов Российской Федерации о взыскании компенсации морального вреда, причиненного незаконным уголовным преследованием,
установил:
дата *1 обратилась в суд с иском к Министерству финансов Российской Федерации о взыскании компенсации морального вреда, причиненного незаконным уголовным преследованием, указав в его обоснование, что в отношении неё следственным отделом * управления Следственного комитета Российской Федерации по * (далее СО по * СУ СК России по *) дата было возбуждено уголовное дело по признакам преступления, предусмотренного частью 2 статьи
293
Уголовного кодекса Российской Федерации. Приговором Поронайского городского суда от дата она была оправдана в связи с отсутствием состава преступления и за ней признано право на реабилитацию. дата апелляционным определением Сахалинского областного суда приговор Поронайского городского суда оставлен без изменения. Таким образом, она в период с дата по дата (более двух лет) подвергалась незаконному уголовному преследованию. Сотрудниками СО по * СУ СК России по * о возбуждении в отношении неё уголовных дел уведомлялись заинтересованные лица и активно распространялась информация о принятых решениях в средствах массовой информации, что необоснованно дискредитировало её, являющуюся сотрудником администрации Поронайского городского округа перед сотрудниками и жителями небольшого *. В связи с незаконными действиями СО по * СУ СК России по * у нее ухудшилось состояние здоровья и обострился диагноз псориаз, в связи с чем она проходила лечение. Также ей была избрана мера пресечения в виде подписки о невыезде и надлежащем поведении, которой она и ее двое несовершеннолетних детей были ограничены в свободном придвижении за пределами *. При подготовке к судебным заседаниям, она уделяла внимание изучению законодательства, вместо проведения времени с семьей. Она находилась в страхе и нарушился сон. Просит взыскать компенсацию морального вреда, причиненного незаконным уголовным преследованием и привлечением к уголовной ответственности, в сумме 800000 рублей.
Истец *1 в судебном заседании исковые требования поддержала, настаивала на их удовлетворении.
Представитель Министерства Финансов Российской Федерации в судебное заседание не явился, просил о рассмотрении дела в его отсутствие, в возражениях на иск просит в отказать в его удовлетворении в полном объеме.
Представители третьих лиц на стороне ответчика * прокуратуры *3, СУ СК России по * *4 полагали, что иск не подлежит удовлетворению в заявленном размере, по основаниям, указанным в возражениях на иск, из которых следует, что доказательств причинения морального вреда в заявленном объеме истцом не представлено, в связи с чем сумма компенсации морального вреда является явно завышенной и не соответствующей принципам разумности и справедливости. При избрании меры пресечения в виде подписки о невыезде и надлежащем поведении, действующим законодательством установлена возможность, на основании ходатайства заинтересованного лица, с разрешения следователя (на стадии предварительного расследования), суда (на стадии судебного разбирательства) покидать место постоянного или временного проживания, о чем *1 знала, но к следователю с данным вопросом не обращалась, таким образом, говорить о нарушении ее права на свободу передвижения нельзя. Наличие публикаций и указание на болезнь не могут служить бесспорным основанием для подтверждения наличия причинно-следственной связи между предварительным следствием и ухудшением состояния ее здоровья.
Выслушав присутствующие стороны, изучив материалы дела и оценив доказательства в их совокупности, суд приходит к следующему.
В соответствии со статьей 133 Уголовно-процессуального кодекса Российской Федерации (далее УПК РФ) право на реабилитацию включает в себя право на возмещение имущественного вреда, устранение последствий морального вреда и восстановление в трудовых, пенсионных, жилищных и иных правах. Вред, причиненный гражданину в результате уголовного преследования, возмещается государством в полном объеме независимо от вины органа дознания, дознавателя, следователя, прокурора и суда.
Право на реабилитацию, в том числе право на возмещение вреда, связанного с уголовным преследованием, имеют: подсудимый, в отношении которого вынесен оправдательный приговор; подсудимый, уголовное преследование в отношении которого прекращено в связи с отказом государственного обвинителя от обвинения; подозреваемый или обвиняемый, уголовное преследование в отношении которого прекращено по основаниям, предусмотренным пунктами 1, 2, 5 и 6 части первой статьи 24 и пунктами 1 и 4 - 6 части первой статьи
27 УПК РФ
; осужденный - в случаях полной или частичной отмены вступившего в законную силу обвинительного приговора суда и прекращения уголовного дела по основаниям, предусмотренным пунктами 1 и 2 части первой статьи 27 настоящего Кодекса; лицо, к которому были применены принудительные меры медицинского характера, - в случае отмены незаконного или необоснованного постановления суда о применении данной меры. Право на возмещение вреда в порядке, установленном настоящей главой, имеет также любое лицо, незаконно подвергнутое мерам процессуального принуждения в ходе производства по уголовному делу.
По смыслу положений пункта 2 статьи
1070 ГК РФ
, причиненный гражданину или юридическому лицу в результате незаконной деятельности органов дознания, предварительного следствия, прокуратуры, не повлекшей последствий, предусмотренных пунктом 1 статьи
1070 ГК РФ
, возмещается по основаниям и в порядке, которые предусмотрены статьей
1069 ГК РФ
.
Согласно статье
1069 ГК РФ
вред, причиненный гражданину или юридическому лицу в результате незаконных действий (бездействия) государственных органов, органов местного самоуправления либо должностных лиц этих органов, в том числе в результате издания не соответствующего закону или иному правовому акту акта государственного органа или органа местного самоуправления, подлежит возмещению. Вред возмещается за счет соответственно казны Российской Федерации, казны субъекта Российской Федерации или казны муниципального образования.
Согласно статье
1071 ГК РФ
в случаях, когда причиненный вред подлежит возмещению за счет казны Российской Федерации, казны субъекта Российской Федерации или казны муниципального образования, от имени казны выступают соответствующие финансовые органы.
В силу статьи
150 ГК РФ
к объектам гражданских прав относятся нематериальные блага такие как жизнь и здоровье, достоинство личности, личная неприкосновенность, принадлежащие гражданину от рождения или в силу закона, неотчуждаемы и непередаваемы другим способом. Одним из способов защиты гражданских прав, предусмотренных статьей
12 ГК РФ
, является компенсация морального вреда.
Согласно статье
151 ГК РФ
, если гражданину причинен моральный вред (физические или нравственные страдания) действиями, нарушающими его личные неимущественные права либо посягающими на принадлежащие гражданину другие нематериальные блага, а также в других случаях, предусмотренных законом, суд может возложить на нарушителя обязанность денежной компенсации указанного вреда. При определении размеров компенсации морального вреда суд принимает во внимание степень вины нарушителя и иные заслуживающие внимания обстоятельства. Суд должен также учитывать степень физических и нравственных страданий, связанных с индивидуальными особенностями лица, которому причинен вред.
В соответствии со статьей
1100 ГК РФ
компенсация морального вреда осуществляется независимо от вины причинителя вреда в случаях, когда вред причинен гражданину в результате его незаконного осуждения, незаконного привлечения к уголовной ответственности, незаконного применения в качестве м еры пресечения заключения под стражу или подписки о невыезде, незаконного наложения административного взыскания в виде ареста или исправительных работ.
Размер компенсации морального вреда в силу требований статьи
1101 ГК РФ
определяется судом в зависимости от характера причиненных потерпевшему физических и нравственных страданий, а также степени вины причинителя вреда в случаях, когда вина является основанием возмещения вреда. При определении размера компенсации вреда должны учитываться требования разумности и справедливости. Характер физических и нравственных страданий оценивается судом с учетом фактических обстоятельств, при которых был причинен моральный вред, и индивидуальных особенностей потерпевшего.
В соответствии с разъяснениями, содержащимися в пункте 8 Постановления Пленума Верховного Суда Российской Федерации от дата * «Некоторые вопросы применения законодательства о компенсации морального вреда», при рассмотрении требований о компенсации причиненного гражданину морального вреда необходимо учитывать, что размер компенсации зависит от характера и объема причиненных истцу нравственных или физических страданий, степени вины ответчика в каждом конкретном случае, иных заслуживающих внимания обстоятельств, и не может быть поставлен в зависимость от размера удовлетворенного иска о возмещении материального вред, убытков и других материальных требований. При определении размера компенсации вреда должны учитываться требования разумности и справедливости.
Пунктом 21 Постановления Пленума Верховного Суда Российской Федерации от дата * «О практике применения судами норм главы 18 Уголовно-процессуального кодекса Российской Федерации, регламентирующих реабилитацию в уголовном судопроизводстве» разъяснено, что при определении размера денежной компенсации морального вреда реабилитированному судам необходимо учитывать степень и характер физических и нравственных страданий, связанных с индивидуальными особенностями лица, которому причинен вред, иные заслуживающие внимания обстоятельства, в том числе продолжительность судопроизводства, длительность и условия содержания под стражей, вид исправительного учреждения, в котором лицо отбывало наказание, и другие обстоятельства, имеющие значение при определении размера компенсации морального вреда, а также требования разумности и справедливости.
Из материалов дела следует, что СО по * СУ СК России по * в отношении *1 дата было возбуждено уголовное дело * по признакам преступления, предусмотренного частью 2 статьи
293
Уголовного кодекса Российской Федерации.
Приговором Поронайского городского суда от дата *1 была оправдана в связи с отсутствием состава преступления и за ней признано право на реабилитацию.
Апелляционным определением Сахалинского областного суда от дата приговор Поронайского городского суда оставлен без изменения и вступил в законную силу.
*1 была избрана мера пресечения в виде подписки о невыезде и надлежащем поведении.
Принимая во внимание изложенное, суд находит установленным факт незаконного уголовного преследования и привлечения к уголовной ответственности *1, а, следовательно, наличие у нее правовых оснований для взыскания компенсации морального вреда.
Определяя размер компенсации морального вреда, суд учитывает фактические обстоятельства, при которых был причинен моральный вред, а именно, то обстоятельство, что уголовное преследование длилось на протяжении длительного времени – более двух лет с дата по дата , применение к *1 процессуального принуждения, степень и глубину нравственных страданий истицы, которая трудилась в тот период и по настоящее время на государственной должности в администрации Поронайского городского округа, наличие публикаций в средствах массовой информации, последствием которого явилось причинение ущерба ее деловой репутации, а также не позитивное отношение сослуживцев в этой связи; имеющую на иждивении несовершеннолетних детей, которую суд признает значительной, так как сам по себе факт уголовного преследования, сопряженный с проведением следственных мероприятий по делу и нахождением под бременем ответственности за преступление, которое ею не совершалось, влечет переживания, повышенную психологическую нагрузку, обострение эмоционального состояния *1, вследствие чего она вынуждена была неоднократно проходить лечение с диагнозом, резвившимся на нервной почве, чего ранее с ней не происходило.
Кроме того, суд учитывает обязанность по соблюдению предусмотренных законом требований разумности и справедливости обеспечения баланса частных и публичных интересов с тем, чтобы выплата компенсации морального вреда одним категориям граждан не нарушала бы права других категорий граждан, учитывая, что казна Российской Федерации формируется в соответствии с законодательством за счет налогов, сборов и платежей, взимаемых с граждан и юридических лиц, которые распределяются и направляются в том числе как возмещение вреда, причиненного государственными органами.
С учетом требований разумности и справедливости, суд полагает необходимым взыскать в пользу истца с Министерства финансов Российской Федерации за счет казны Российской Федерации, денежную компенсацию морального вреда в размере 70000 рублей.
На основании изложенного, руководствуясь статьями
194
-
199 ГПК РФ
, суд
решил:
Исковые требования по исковому заявлению *1 к Министерству финансов Российской Федерации в лице Управления федерального казначейства по * о взыскании компенсации морального вреда, причиненного незаконным уголовным преследованием и привлечением к уголовной ответственности, удовлетворить частично.
Взыскать с Министерства Финансов Российской Федерации за счёт средств казны Российской Федерации в пользу *1 компенсацию морального вреда, причиненного незаконным уголовным преследованием и привлечением к уголовной ответственности, в размере 70000 (семьдесят тысяч) рублей 00 копеек.
В остальной сумме иска по данному требованию – отказать.
На решение может быть подана апелляционная жалоба, апелляционное представление в Сахалинский областной суд через Поронайский городской суд в течение месяца со дня его вынесения в окончательном виде.
Судья Поронайского городского суда: подпись
Копия верна, судья Л.В. Домникова
Суд:
Поронайский городской суд (Сахалинская область)
(подробнее)
Судьи дела:
Домникова Любовь Викторовна (судья)
(подробнее)
Последние документы по делу:
Решение от 27 мая 2020 г. по делу № 2-225/2020
Решение от 15 апреля 2020 г. по делу № 2-225/2020
Решение от 8 апреля 2020 г. по делу № 2-225/2020
Показать все документы по этому делу
Судебная практика по:
Моральный вред и его компенсация, возмещение морального вреда
Судебная практика по применению норм ст. 151, 1100 ГК РФ
Халатность
Судебная практика по применению нормы ст. 293 УК РФ

"""[:60]

'\nДело *\nР Е Ш Е Н И Е\nИ М Е Н Е М Р О С С И Й С К О Й Ф Е Д '

В юридических организациях проблема заключается в масштабе и усилиях. Объем документов в большинстве юридических организаций, даже крупных, далеко не тот, для которого были разработаны инструменты ИИ. Кроме того, проверка и сборка таких наборов данных, а также проверка работоспособности продукта после обучения на таких коллекциях могут занимать чрезвычайно много времени. 

Он поддерживает три основных варианта использования: семантический поиск, классификацию и генерацию текста.

Классификация протоколов аварийных ситуаций для страховой компании.

Берем готовую версию модели и дообучаем на корпусе текстов юр. характера.

1-й этап - раздобыть корпус тексов юридического характера. Для этого у нас есть:

* Министерство юстиции РФ (minjust.gov.ru) - 419 документов
* minjust-prikaz.consultant.ru - 912 документов

Можно использовать другие ресурсы.

2-й этап - дообучить.
Это требует ресурсы и понимание технологии.
Берем готовую модель и дообучаем.

Нужно понять, как запустить и как дообучить.

 Генерация текста — самая полезная и впечатляющая функция GPT-3: ее можно использовать для чего угодно: от создания полностью рабочего кода до постов в блогах и даже табулатуры для гитары. Примечательно, что программа обучается без какого-либо контроля со стороны человека. Получая множество различных данных из Интернета, включая Википедию, сообщения Reddit, всевозможные блоги, новостные статьи и даже фанфики, он находит шаблоны на основе статистических закономерностей, которые затем использует для выполнения текстовых подсказок.

Нейронная сеть следует набору правил для удовлетворения конкретных потребностей, и полоса креативности является одной из ее наиболее заметных настроек. Пользователи могут либо выбрать небольшую креативность, чтобы сузить алгоритм обучения до предоставленных материалов и получить очень структурированную и основанную на фактах информацию, либо установить высокую креативность и наблюдать, как ИИ делает свои собственные предположения и генерирует творческий (хотя и не обязательно правдивый) контент.

Благодаря своей замечательной способности обобщать и расширять содержание, модель может превратить длинный текст в краткий бриф и наоборот.

Классификация метаданных из любого юридического документа.

GPT-3 автоматизирует самую скучную часть юридической работы, такую ​​как заполнение запросов на допуск (RFA)

Ввод для GPT-3 не должен превышать 1000 символов.

Как правило, языковая модель, такая как GPT-3, адаптируется к конкретной задаче, такой как ответы на вопросы о документе или краткое изложение документа, с помощью метода, называемого «точной настройкой». Здесь вы изменяете архитектуру, чтобы заставить ее выполнять новую задачу, а затем продолжаете обучать ее на 100–1000 новых примерах, чтобы «настроить» ее на новую задачу.

Вместо этого в GPT-3 OpenAI использует более простую технику, которая настраивает модель на имитацию определенного поведения, предоставляя ей всего несколько примеров. Вес самой модели остается неизменным.

Удивительным результатом является то, что благодаря своему увеличенному размеру GPT-3 часто может приблизиться или соответствовать современным результатам, которые были достигнуты путем точной настройки на меньших моделях без точной настройки.

Мы можем быть уверены, что GPT-3 не видел приватных данных; это может означать, что он будет менее эффективен для типов документов, которые обычно не являются общедоступными.

Парсим сканы документов с сайта министерства юстиций.

Храним метаданные по документам в sqlite.

Распознаем текст со сканов. 

Учим GPT3 классифицировать юридические документы.

К счастью, API GPT-3 дает нам несколько ручек для настройки, чтобы почти полностью исключить предсказания за пределами допустимого диапазона. В этом сообщении блога мы рассмотрим два из них:

Регулировка temperature.
Установка logit_bias.
Регулировка температуры
Параметр temperatureдовольно простой. Документация GPT-3 объясняет, как temperatureэто работает: «Более высокие значения означают, что модель будет подвергаться большему риску. Попробуйте 0,9 для более творческих приложений и 0 (выборка argmax) для тех, у которых есть четко определенный ответ». Для задач классификации текста обычно требуется установить это значение равным 0.

Несмотря на то, что установка temperatureзначения 0 значительно уменьшает количество прогнозов, выходящих за пределы допустимого диапазона, она не устраняет их полностью. Чтобы сделать еще один шаг вперед, вы можете использовать logit_biasпараметр (подробнее об этом чуть позже).

Параметр logit_bias
Токенизация: как GPT-3 видит текст
Чтобы понять, как logit_biasможно использовать для создания белого списка меток, нам сначала нужно сделать шаг назад и объяснить, как GPT-3 интерпретирует текст. GPT-3 видит текст не как строку символов или слов, а как список «идентификаторов токенов». Идентификатор каждого токена — это уникальный номер, представляющий собой короткую последовательность символов (в среднем каждый токен соответствует примерно 4 символам английского текста). Процесс преобразования текста в токены основан на частоте определенных комбинаций символов. Распространенным словам, таким как «кошка» и «мяч», скорее всего, будет присвоен собственный уникальный токен, в то время как менее распространенные слова, такие как «учреждение», скорее всего, будут разделены на более чем один токен (например, «установить», «лишиться», «мент»).
https://medium.com/edge-analytics/getting-the-most-out-of-gpt-3-based-text-classifiers-part-one-797460a5556e

## Корпус текстов
Банк данных "Приказы и распоряжения Министерства юстиции Российской Федерации" - https://minjust-prikaz.consultant.ru/

Министерство юстиции Российской Федерации - https://minjust.gov.ru/ru/documents/

СудАкт: Судебные и нормативные акты РФСудебные и нормативные акты РФ - https://sudact.ru/regular/#doc_list

Государственная автоматизированная система Российской Федерации «Правосудие» (Информация о деятельности федеральных судов общей юрисдикции) - https://sudrf.ru/index.php?id=300

Архив решений арбитражных судов и судов общей юрисдикции - https://sudrf.cntd.ru/

На настоящий момент в России в открытом доступе находятся датасеты, созданные на основе решений судов по уголовным делам. Так, проект Достоевский содержит открытые данные об уголовных делах в России с 2009 года. Создатели проекта собирают, обрабатывают и визуализируют датасеты из официальной статистики Судебного департамента при Верховном суде РФ. - https://beta.dostoevsky.io/ru/full

 С 2011 года предполагается официальное опубликование правовых актов на Официальном интернет-портале правовой информации ( pravo.gov.ru ). Сейчас он включает и федеральное, и региональное, и муниципальное законодательство, но информация не полная. Однако такие документы бывают только в графическом формате (отсканированный TIFF или PDF без текстового слоя).
 
 https://www.clarin.eu/resource-families/legal-corpora

## Источники

Автодополнение текста на естественном языке c использованием глубокого обучения, Бакалаврская работа, Менчуков Павел Александрович, 